In [13]:
from collections import Counter
import pandas as pd
import gensim
from konlpy.tag import Komoran
import sentencepiece as spm
import matplotlib.pyplot as plt
import re

plt.rc('font', family='malgun')
%config InlineBackend.figure_format = 'retina'

### 토큰화한 데이터를 word2vec으로 표현하기

In [14]:
memo_data = pd.read_csv("car_inspect_info.tsv", sep='\t', lineterminator='\r')
memo_data.head()

,inspect_id,car_id,inspect_at,inspect_type,price,description
0,\n2504316,8421.0,2020-03-09,타이어수리(후우),6600.0,조수석뒤타이어 현장펑크수리
1,\n2401681,6654.0,2020-01-07,미션오일,120000.0,단양애니카_미션오일 교환
2,\n2384487,8563.0,2020-01-14,점화플러그,60000.0,당진세계모터스_점화플러그 교환
3,\n2384485,8563.0,2020-01-14,브레이크액,50000.0,당진세계모터스-브레이크오일 교환
4,\n2364371,6424.0,2020-01-06,브레이크액,40000.0,당진세계모터스_브레이크오일 교환


* inspect_id : 수리요청 고유 번호
* car_id : 자동차 고유 번호
* inspect_at : 수리요청 발생 시간
* inspect_type : 수리요청 카테고리
* price : 수리 비용
* description : 수리 관련 메모

In [15]:
# inspect_id, car_id, price는 사용하지 않을 예정이니 지움
memo_data = memo_data.drop(columns=['inspect_id', 'car_id', 'price'])

In [16]:
memo_data[memo_data.isna().any(axis=1)]

,inspect_at,inspect_type,description
18,NaN,NaN,NaN
20,NaN,NaN,NaN
22,NaN,NaN,NaN
58,NaN,NaN,NaN
169,NaN,NaN,NaN
...,...,...,...
272856,NaN,NaN,NaN
272858,NaN,NaN,NaN
272860,NaN,NaN,NaN
272862,NaN,NaN,NaN


In [17]:
memo_data = memo_data.dropna() # drop nan row
memo_data.head()

,inspect_at,inspect_type,description
0,2020-03-09,타이어수리(후우),조수석뒤타이어 현장펑크수리
1,2020-01-07,미션오일,단양애니카_미션오일 교환
2,2020-01-14,점화플러그,당진세계모터스_점화플러그 교환
3,2020-01-14,브레이크액,당진세계모터스-브레이크오일 교환
4,2020-01-06,브레이크액,당진세계모터스_브레이크오일 교환


In [18]:
# 토큰화
komoran = Komoran()

# def to_token(df, col):
# 
#     morph_description = []
#     for idx, row in df.iterrows():
#         memo = row[col]
# 
#         try: 
#             morph_out = komoran.morphs(memo.replace('_',' ').replace('-',' ').replace('/',' ').replace('ㄴ',' ').replace('#',' ').replace('=',' ').replace(')',' ) ').replace('(',' ( '))
#             morph_description.append(' '.join(morph_out))
# 
#         except: 
#             df.drop(idx, inplace=True)
# 
#     df["subword_description"] = morph_description 
#     return df



def to_token(df, col):
    
    morph_description = []
    for idx, row in df.iterrows():
        memo = row[col]

        try:
            # 정규식을 사용하여 특수 문자를 공백으로 대체
            morph_out = komoran.morphs(re.sub(r'[^a-zA-Z0-9가-힣\s]', ' ', memo))
            morph_description.append(' '.join(morph_out))
            
        except:
            df.drop(idx, inplace=True)

    df["subword_description"] = morph_description
    return df

memo_data = to_token(memo_data, "description")
memo_data.head()

,inspect_at,inspect_type,description,subword_description
0,2020-03-09,타이어수리(후우),조수석뒤타이어 현장펑크수리,조수석 뒤 타이어 현장 펑크 수리
1,2020-01-07,미션오일,단양애니카_미션오일 교환,단양 애니 카 미션 오일 교환
2,2020-01-14,점화플러그,당진세계모터스_점화플러그 교환,당진 세계 모터스 점화 플러그 교환
3,2020-01-14,브레이크액,당진세계모터스-브레이크오일 교환,당진 세계 모터스 브레이크 오일 교환
4,2020-01-06,브레이크액,당진세계모터스_브레이크오일 교환,당진 세계 모터스 브레이크 오일 교환


In [19]:
len(memo_data)

254064

In [20]:
w2v_train = list(memo_data['subword_description'])
w2v_train = [line.split() for line in w2v_train]

print(len(w2v_train))

254064


In [10]:
model = gensim.models.word2vec.Word2Vec(w2v_train, vector_size=300, window=5, min_count=1, epochs=10)
# save Word2Vec
model.save('vec300_w2v.model')
# load Word2Vec
model = gensim.models.Word2Vec.load('vec300_w2v.model')

In [11]:
# check word most similarity
model.wv.most_similar('타이어', topn=10)

[('디스크', 0.40206199884414673),
 ('파스', 0.39008191227912903),
 ('마모', 0.3850942850112915),
 ('도어스커프트립', 0.37463974952697754),
 ('모선', 0.3715605139732361),
 ('iq', 0.3695164620876312),
 ('부츠', 0.36856889724731445),
 ('허브', 0.3545205295085907),
 ('드럼', 0.3545028865337372),
 ('송정', 0.3533509075641632)]

In [13]:
model.wv.most_similar('엔진', topn=10)

[('소거', 0.5868825912475586),
 ('배기가스', 0.5463823676109314),
 ('스캐너', 0.5398175716400146),
 ('2815401', 0.5196344256401062),
 ('DPF', 0.5039170384407043),
 ('에어백', 0.49814853072166443),
 ('스캔너', 0.496612012386322),
 ('과거', 0.485440194606781),
 ('스패너', 0.46449461579322815),
 ('진단', 0.4592687487602234)]

In [14]:
# 상위 10개의 단어 중 하위에 위치한 단어는 유사성이 부족한 것을 볼 수 있음.
# 이 부분을 보완하기 위해 top 10의 유사도 값을 모두 더하여 평균내고, 이 평균값을 기준(Threshold)로 정하여 기준보다 상위에 위치한 값을 사용하도록 할 수 있음

def exclusion(word_list):
    count = sum([score for _, score in word_list])
    avg = count / len(word_list)
  
    up_word = []
    for w, s in word_list:
        if s > avg:
            up_word.append(w)
  
    return up_word

In [15]:
def load_car_dic():
    car_dic_file = open('carDic.txt','r',encoding='utf8')
    car_dic = set(car_dic_file.read().split('\n'))
    return car_dic

car_dic = load_car_dic()
print(len(car_dic)) #1433

1433


In [16]:
memo_data = pd.read_csv('car_inspect_info.tsv', sep='\t', lineterminator='\r')
memo_data = memo_data.drop(columns=['inspect_id', 'car_id', 'price']) # drop columns
memo_data = memo_data.dropna() # drop nan row
memo_data.head()

,inspect_at,inspect_type,description
0,2020-03-09,타이어수리(후우),조수석뒤타이어 현장펑크수리
1,2020-01-07,미션오일,단양애니카_미션오일 교환
2,2020-01-14,점화플러그,당진세계모터스_점화플러그 교환
3,2020-01-14,브레이크액,당진세계모터스-브레이크오일 교환
4,2020-01-06,브레이크액,당진세계모터스_브레이크오일 교환


In [17]:
#sentenctePiece를 학습하기 위해서는 별도의 text 파일이 필요함
#따라서 이를 만드는 부분
with open('car_info_description.txt','w',encoding='utf8') as w, open('KCC150_100k.txt','r',encoding='utf8') as corpus:
    w.write('\n'.join(memo_data['description']))
    for idx, snt in enumerate(corpus.readlines()):
        w.write(snt)